In [1]:
from qutip import *
from qutip.qip.operations import *
from math import *
import pandas as pd
from numpy import *


c:\Users\Navid Elyasi\AppData\Local\Programs\Python\Python310\lib\site-packages\qutip\__init__.py:66: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


# Automatic Derivation

In [2]:
#Defining The State of the Battery 
Battery = basis(2, 0)
Ancilla = basis(2, 0)
def initialize():
    Joint_State = tensor(basis(2, 0), basis(2, 0))
    Density = Joint_State*Joint_State.dag()
    return Density 
    


#Projective Measurements
Plus = 1/sqrt(2)*(basis(2, 0)+ basis(2, 1))
Plus = Plus*Plus.dag()
Minus = 1/sqrt(2)*(basis(2, 0)- basis(2, 1))
Minus = Minus*Minus.dag()
PP = tensor(qeye(2), Plus)
PM = tensor(qeye(2), Minus)

#Charging Operator
# Initially we have to define the Hamiltonian 
omega = 1
kappa = 1

H = omega * (tensor(sigmax(), qeye(2))) + kappa * (
tensor(sigmam(), sigmap()) + tensor(sigmap(), sigmam()))
# Generating the Unitary gate which is responsible for charging the battey and correlating it with the ancilla 
M_Unitary = (-1j * H).expm()

#Define the number of Steps 
Steps = 6

In [3]:
from itertools import product

def steps_sample_space(steps):
    # Find out the sample space 
    outcomes = list(product([0, 1], repeat=steps))

    #Save the sample space in a list
    sample_space = [list(outcome) for outcome in outcomes]

    #Consider + as 0 and - as 1
    sample_space = [[1 if outcome == 1 else 0 for outcome in step] for step in sample_space]

    #Convert every trajectory from binary to decimal and save it in another list
    decimal_trajectories = [int(''.join(map(str, step)), 2) for step in sample_space]
    
    #Sort both lists together in ascending order
    sorted_lists = sorted(zip(sample_space, decimal_trajectories), key=lambda x: x[1])
    sorted_sample_space, sorted_decimal_trajectories = zip(*sorted_lists)
    for items in sorted_sample_space:
        items.reverse()


    return sorted_sample_space, sorted_decimal_trajectories

In [4]:
#Finding the Trajectories and save the in the following list 
Trajectories, decimal = steps_sample_space(Steps) 

print(Trajectories)
print(decimal)

([0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0], [1, 1, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 0], [1, 0, 1, 1, 0, 0], [0, 1, 1, 1, 0, 0], [1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0], [0, 1, 1, 0, 1, 0], [1, 1, 1, 0, 1, 0], [0, 0, 0, 1, 1, 0], [1, 0, 0, 1, 1, 0], [0, 1, 0, 1, 1, 0], [1, 1, 0, 1, 1, 0], [0, 0, 1, 1, 1, 0], [1, 0, 1, 1, 1, 0], [0, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 1], [1, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 1], [1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 1], [1, 1, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1], [0, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1], [0, 0, 1, 1, 0, 1], [1, 0, 1, 1, 0, 1], [0, 1, 1, 1, 0, 1], [1, 1, 1, 1, 0, 1], [0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 1, 1],

In [5]:
#This function will be used further to save the data recieved for each arbitrary step
import pandas as pd

def save_lists_to_excel(Trajectory_Save, Battery_Density, Unitary_Matrix, excel_file_path):
    # Create a DataFrame with the two lists as two rows
    df = pd.DataFrame([Trajectory_Save, Battery_Density, Unitary_Matrix])

    # Save the DataFrame to an Excel file
    df.to_excel(excel_file_path, index=False, header=False)



In [6]:
# The following function will be used to find out the unitary matrix for leading us to passive state 

def find_unitary(Theta, Phi):
    U = None
    U = array([[ cos(Theta) , -exp(-1j*Phi)*sin(Theta)], [exp(1j*Phi)*sin(Theta), cos(Theta)]])
    Unitary = Qobj(U)
    return Unitary

In [7]:
#Staring Loop Process To save density matix for each steps 
from cmath import *
excel_file_path = 'output.xlsx' #Saving path of the Output Data 

# Initialize an empty list to store the data
data_list = []

for Trajectory in Trajectories:
    Density = initialize()
    for Measure_Type in Trajectory:
        Evolved = M_Unitary*Density*M_Unitary.dag()   #Charge and interact the Qubit with a specefic \theta in each step
        if Measure_Type == 0:
            Meas_Den_P = PP*Evolved*PP.dag()
            Probabilty_P = Meas_Den_P.tr()
            Meas_Den_P = Meas_Den_P/Probabilty_P
            P_Den = Meas_Den_P.ptrace(0)
            Battery = P_Den
            Density = tensor(Battery, Ancilla*Ancilla.dag())
        if Measure_Type == 1:
            Meas_Den_M = PM*Evolved*PM.dag()
            Probabilty_M = Meas_Den_M.tr()
            Meas_Den_M = Meas_Den_M/Probabilty_M
            M_Den = Meas_Den_M.ptrace(0)
            Battery = M_Den
            Density = tensor(Battery, Ancilla*Ancilla.dag())
    #print(Battery)
    theta = arccos(sqrt(Battery[0, 0]))
    #print(theta)
    off = Battery[0, 1]
    phi = log(off/(sin(theta)*cos(theta)))/1j
    
    Unitary = find_unitary(theta, real(phi))
    #print(Unitary)
    print(Unitary*Battery*Unitary.dag())

            

    data_list.append({'Theta': real(theta), 'Phi': real(phi)})


# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data_list)

# Save the DataFrame to an Excel file
csv_file_path = 'output-m.csv'
df.to_csv(csv_file_path, index=False, header=False)

            

    
    

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]
Quantum object: dim